In [1]:
import numpy as np
import sympy as sp
from IPython.display import display
from sympy.abc import L, z, M, alpha, beta, n

In [2]:
Phi_star, M_star, L_star, Phi = sp.symbols('Phi_*, M_*, L_*, Phi', cls=sp.Function)
QLF = sp.Eq(Phi(L, z), (Phi_star(z) / L_star(z)) * ((L / L_star(z))**-alpha + (L / L_star(z))**-beta)**-1)
QLF

Eq(Phi(L, z), Phi_*(z)/(((L/L_*(z))**(-beta) + (L/L_*(z))**(-alpha))*L_*(z)))

In [3]:
sp.Eq(L/L_star(z), 10**(0.4 * (M_star(z) - M)))

Eq(L/L_*(z), 10**(-0.4*M + 0.4*M_*(z)))

In [4]:
QLF_M = sp.Eq(Phi(M, z), Phi(L, z) * 0.4 * sp.log(10) * L)
QLF_M = QLF_M.subs({Phi(L, z): QLF.rhs}).subs({L/L_star(z): 10**(0.4 * (M_star(z) - M))})
QLF_M

Eq(Phi(M, z), 0.4*10**(-0.4*M + 0.4*M_*(z))*Phi_*(z)*log(10)/((10**(-0.4*M + 0.4*M_*(z)))**(-beta) + (10**(-0.4*M + 0.4*M_*(z)))**(-alpha)))

In [5]:
i, k = sp.symbols('i, k')
a = sp.IndexedBase('M_*')
z = sp.IndexedBase('z')
LumEvol = sp.Eq(M_star(z), sp.Sum(a[i] * sp.Product((z - z[k]) / (z[i] - z[k]), (k, 1, n)), (i, 1, n)))
LumEvol

Eq(M_*(z), Sum(M_*[i]*Product((-z[k] + z)/(z[i] - z[k]), (k, 1, n)), (i, 1, n)))

In [6]:
DenEvol = sp.Eq(sp.log(Phi_star(z), 10), sp.Sum(sp.log(Phi_star(z[i]), 10) * sp.Product((z - z[k]) / (z[i] - z[k]), (k, 1, n)), (i, 1, n)))
display(DenEvol)
sp.Eq(Phi_star(z), sp.solve(DenEvol, Phi_star(z))[0])

Eq(log(Phi_*(z))/log(10), Sum(log(Phi_*(z[i]))*Product((-z[k] + z)/(z[i] - z[k]), (k, 1, n))/log(10), (i, 1, n)))

Eq(Phi_*(z), Product(exp(log(Phi_*(z[i]))*Product((-z[k] + z)/(z[i] - z[k]), (k, 1, n))), (i, 1, n)))

In [7]:
QLF_M.subs({M_star(z): LumEvol.rhs})

Eq(Phi(M, z), 0.4*10**(-0.4*M + 0.4*M_*(z))*Phi_*(z)*log(10)/((10**(-0.4*M + 0.4*M_*(z)))**(-beta) + (10**(-0.4*M + 0.4*M_*(z)))**(-alpha)))

In [8]:
LumEvol.rhs.evalf(2)

Sum(M_*[i]*Product((-z[k] + z)/(z[i] - z[k]), (k, 1, n)), (i, 1, n))

In [9]:
from scipy.interpolate import lagrange
from numpy.polynomial import Polynomial
z_m_star = [0.5, 1., 2., 4.]
m_star = [-23.51, -24.64, -26.10, -27.08]

f = lagrange(z_m_star, m_star)
f.coef
f(0.75)

-24.113020833333334

In [10]:
def LagrangePoly(x, order, j, xi=None):
    if xi is None:
        xi = sp.symbols(f'z:{order + 1:d}')

    idx = np.arange(order + 1)
    idx = np.delete(idx, j)
    return sp.prod([(x - xi[l]) / (xi[j] - xi[l]) for l in idx])

In [11]:
L = [LagrangePoly(sp.symbols('z'), order=3, j=i, xi=z_m_star) for i in range(len(z_m_star))]
for basis in L:
    display(basis.expand())

-0.380952380952381*z**3 + 2.66666666666667*z**2 - 5.33333333333333*z + 3.04761904761905

0.666666666666667*z**3 - 4.33333333333333*z**2 + 7.33333333333333*z - 2.66666666666667

-0.333333333333333*z**3 + 1.83333333333333*z**2 - 2.16666666666667*z + 0.666666666666667

0.0476190476190476*z**3 - 0.166666666666667*z**2 + 0.166666666666667*z - 0.0476190476190476

In [12]:
lp = sp.Add(*[m_star[m] * L[m] for m in range(len(L))])
display(lp.simplify())
lp_f = sp.lambdify(sp.symbols('z'), lp)

-0.0600000000000009*z**3 + 0.743333333333322*z**2 - 3.26999999999999*z - 22.0533333333333

In [13]:
np.isclose(lp_f(0.75), f(0.75))

True

In [14]:
f.coef

array([ -0.06      ,   0.74333333,  -3.27      , -22.05333333])

In [15]:
r, m, z, J = sp.symbols('r,m,z,J')
theta, eta, zeta, beta, rc, C = sp.symbols('theta, eta, zeta, beta, r_c, C')
N = sp.symbols('N', cls=sp.Function)(z, m, r, J)
DA = sp.symbols('D_A', cls=sp.Function)(z)
LF = sp.symbols('Phi', cls=sp.Function)(J, z)


model = theta * (1 + z)**eta * (m / 1e15)**zeta * (1 + (r / rc)**2)**(-1.5 * beta + 0.5) * DA**2 * r * LF + C
model

C + r*theta*(1.0e-15*m)**zeta*(z + 1)**eta*(r**2/r_c**2 + 1)**(0.5 - 1.5*beta)*D_A(z)**2*Phi(J, z)

In [16]:
sp.Integral(model, r, J).simplify()

C*J*r + theta*(1.0e-15*m)**zeta*(z + 1)**eta*D_A(z)**2*Integral(r*(r**2/r_c**2 + 1)**0.5/(r**2/r_c**2 + 1)**(1.5*beta), r)*Integral(Phi(J, z), J)

In [17]:
sp.Eq(sp.Integral(N, r, J), sp.Integral(model, r, J))

Eq(Integral(N(z, m, r, J), r, J), Integral(C + r*theta*(1.0e-15*m)**zeta*(z + 1)**eta*(r**2/r_c**2 + 1)**(0.5 - 1.5*beta)*D_A(z)**2*Phi(J, z), r, J))

In [ ]:
R, Jmin, Jmax = sp.symbols('R, J_min, J_max')
sp.integrate(model, (r, 0, R), (J, Jmin, Jmax))

In [ ]:
R, Jmin, Jmax = sp.symbols('R, J_min, J_max')
sp.Integral(C, (r, 0, R), (J, Jmin, Jmax)).doit()

In [ ]:
# Set up the luminosity and density evolution using the fits from Assef+11 Table 2
z_i = [0.25, 0.5, 1., 2., 4.]
m_star_z_i = [-23.51, -24.64, -26.10, -27.08]
phi_star_z_i = [-3.41, -3.73, -4.17, -4.65, -5.77]
m_star = lagrange(z_i[1:], m_star_z_i)
log_phi_star = lagrange(z_i, phi_star_z_i)

In [ ]:
sp.__version__

In [ ]:
R, Jmin, Jmax = sp.symbols('R, J_min, J_max')
sp.Integral(C, (r, 0, R), (J, Jmin, Jmax)).doit()

In [ ]:
# Set up the luminosity and density evolution using the fits from Assef+11 Table 2
z_i = [0.25, 0.5, 1., 2., 4.]
m_star_z_i = [-23.51, -24.64, -26.10, -27.08]
phi_star_z_i = [-3.41, -3.73, -4.17, -4.65, -5.77]
m_star = lagrange(z_i[1:], m_star_z_i)
log_phi_star = lagrange(z_i, phi_star_z_i)

In [ ]:
sp.__version__

In [19]:
R, Jmin, Jmax = sp.symbols('R, J_min, J_max')
sp.Integral(C, (r, 0, R), (J, Jmin, Jmax)).doit()

C*J_max*R - C*J_min*R

In [24]:
# Set up the luminosity and density evolution using the fits from Assef+11 Table 2
z_i = [0.25, 0.5, 1., 2., 4.]
m_star_z_i = [-23.51, -24.64, -26.10, -27.08]
phi_star_z_i = [-3.41, -3.73, -4.17, -4.65, -5.77]
m_star = lagrange(z_i[1:], m_star_z_i)
log_phi_star = lagrange(z_i, phi_star_z_i)

In [25]:
sp.__version__

'1.9'